In [1]:
with open('/home/byron/repos/cmix/data/enwik9', 'r') as file:
    enwik9 = file.read()
articles = enwik9.split("<page>")
enwik9 = ""

In [2]:
import re
first = 0
last = len(articles)
texts = []
m = {}
info_count = 0
image_count = 0
disam_count = 0
geo_count = 0
hndis_count = 0
census_count = 0
for i in range(last-first):
    index = i + first
    start = articles[index].find("<text xml:space=\"preserve\">")
    end = articles[index].find("</text>")
    if (start > 0 and end > 0):
        if (articles[index][start+27] == '#' and (articles[index][start+28] == 'R' or articles[index][start+28] == 'r')):
            continue
        title_start = articles[index].find("<title>")
        title_end = articles[index].find("</title>")
        title = articles[index][title_start+7:title_end]
        content = articles[index][start+27:end]
        info = False
        if (re.search("{{[^}]*\n[^}]*}}", content, flags = re.MULTILINE)):
            info = True
            info_count += 1
        image = False
        if (title.startswith("Image:")):
            image = True
            image_count += 1
        disam = False
        if (title.find("(disambiguation)") >= 0 or re.search("{{[Dd]isam.*}}", content)):
            disam = True
            disam_count += 1
        hndis = False
        if (re.search("{{hndis", content)):
            hndis = True
            hndis_count += 1
        geo = False
        if (content.find("{{geodis}}") >= 0):
            geo = True
            geo_count += 1
        census = False
        if (content.find("According to the [[United States Census Bureau]],") >= 0 or content.find("According to the [[U.S. Census Bureau]],") >= 0 or content.find("As of the [[2000]] census,") >= 0 or content.find("According to the [[2000]] census, ") >= 0 or content.find("According to the [[United States Census Bureau|U.S. Census Bureau]],") >= 0 or content.find("According to the [[US Census Bureau]], ") >= 0 or content.find("According to the U.S. Census,") >= 0 or content.find("As of the 2000 ") >= 0 or content.find("As of the census of 2000,") >= 0):
            census = True
            census_count += 1
        m[index - 1] = title + "\t" + str(info) + "\t" + str(image) + "\t" + str(disam) + "\t" + str(geo) + "\t" + str(census) + "\t" + str(hndis)
print("info: " + str(info_count))
print("image: " + str(image_count))
print("disam: " + str(disam_count))
print("geo: " + str(geo_count))
print("census: " + str(census_count))
print("hndis: " + str(hndis_count))

info: 21546
image: 13908
disam: 7473
geo: 325
census: 36519
hndis: 157


In [3]:
f = open("analyze.tsv", "w")
data = []
with open("final-clustered-2154-4k-160ktsne-rev.txt", "r") as out:
    lines = out.readlines()
    for i in range(len(lines)):
        data.append([])
        key = int(lines[i])
        f.write(m[key] + "\n")
        s = m[key].split("\t")
        data[i].append(i)
        data[i].append(key)
        for j in range(len(s)):
            data[i].append(s[j])
f.close()

In [4]:
def compare(item1, item2):
    if item1[8] != item2[8]: # hndis
        if (item1[8] == 'True'):
            return 1
        return -1
    if item1[6] != item2[6]: # geo
        if (item1[6] == 'True'):
            return 1
        return -1
    if item1[5] != item2[5]: # disam
        if (item1[5] == 'True'):
            return 1
        return -1
    if item1[4] != item2[4]: # image
        if (item1[4] == 'True'):
            return 1
        return -1
    if item1[7] != item2[7]: # census
        if (item1[7] == 'True'):
            return 1
        return -1
    return item1[0] - item2[0]

from functools import cmp_to_key
sort = sorted(data, key=cmp_to_key(compare))

In [5]:
f = open("analyze.tsv", "w")
for i in range(len(sort)):
    f.write('\t'.join(str(x) for x in sort[i]) + "\n")
f.close()

In [6]:
f = open("final.txt", "w")
for i in range(len(sort)):
    f.write(str(sort[i][1]) + "\n")
f.close()